## Example: Working with models from CLI

HydroMT has the following high-level functionality from the Command Line Interface (CLI) for setting up models from raw data or adjusting models:

- **building** a model: building a model from scratch.
- **updating** a model: adding or changing model components of an existing model.
- **clipping** a model: changing the spatial domain of an existing model (e.g. select subbasins from a larger model).

Here we show how to build and update a hypothetical distributed model from the command line interface (CLI) based on the generic HydroMT **grid_model**.

Lets first check which models are available in our environment:

In [ ]:
!hydromt --models

### Build a model from CLI

To build a model you always follow the next four steps. 

1. Prepare or use a pre-defined data catalog with all the required data sources
2. Define your model region, see the overview of [model region options](https://deltares.github.io/hydromt/latest/user_guide/model_region.html).
3. Prepare a model configuration which describes the complete pipeline to build your model, see [preparing a model configuration](https://deltares.github.io/hydromt/latest/user_guide/model_config.html). 
4. Build you model using the CLI or Python interface

Here we focus steps 2-4 and use data from the [predefined **aritifact_data** data catalog](https://deltares.github.io/hydromt/latest/user_guide/data_existing_cat.html)

Using the **hydromt build** method we can setup a complete model from scratch. Let's get an overview of the method and its arguments. 

Note the required `MODEL` (i.e. name of the model), `MODEL_ROOT` (i.e. folder where to save the model) arguments.
As of version v0.7.0 the `REGION` (i.e. area of interest) argument is optional. This argument can be used by adding -r or --region flag.

In [ ]:
!hydromt build --help

In [ ]:
!hydromt build grid_model ./tmp_grid_model --region "{'bbox': [11.70, 45.35, 12.95, 46.70]}" -vv

The example above means the following: run **hydromt build** with:

- `grid_model`: i.e. build a generic GridModel instance
- `./tmp_grid_model`: output model folder
- `--region "{'bbox': [11.70, 45.35, 12.95, 46.70]}"`: set the region of interest using a bounding box defined by its [xmin, ymin, xmax, ymax] coordinates (in WGS84)
- `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

As we did not specify a hydromt configuration, besides the hydromt.log file, an empty model simulation configuration file, only the model region (area of interest) has been defined and is saved in the geoms folder. To build a complete model we need the use a hydromt configuration ini-file. 

In [ ]:
# print MODEL_ROOT folder
import os


def print_dir(root):
    for path, _, files in os.walk(root):
        print(path)
        for name in files:
            if name.endswith(".xml"):
                continue
            print(f" - {name}")


print_dir("tmp_grid_model")

The **configuration file** exists of available  **setup_methods** for your model, listed in the order of execution. Most methods start with reading input data using the DataAdapter, transforming the data using workflows (e.g. reprojection, reclassification, aggregation, etc...) and adding the new model data to the right model component. An overview of the available methods can be found in the online API reference for the [GridModel](https://deltares.github.io/hydromt/latest/api.html#gridmodel), [LumpedModel](https://deltares.github.io/hydromt/latest/api.html#lumpedmodel), and [MeshModel](https://deltares.github.io/hydromt/latest/api.html#meshmodel)

Note that these methods for the generic model classes are still quite limited. To get an idea of potential setup_ methods, checkout the [model plugins](https://deltares.github.io/hydromt/latest/plugins.html)

The configuration ini file can be passed to the hydromt build method using the `-i` flag.

In [ ]:
# check the content of the ini file (for sake of the example only)
fn_ini = "grid_model_build.ini"
with open(fn_ini, "r") as f:
    txt = f.read()
print(txt)

In [ ]:
!hydromt build grid_model ./tmp_grid_model1 -r "{'bbox': [11.70, 45.35, 12.95, 46.70]}" -i grid_model_build.ini -d artifact_data -d data/vito_reclass.yml -vv

The example above means the following: run **hydromt build** with:

- `grid_model`: i.e. build a generic GridModel instance
- `./tmp_grid_model1`: output model folder
- `-r "{'bbox': [11.70, 45.35, 12.95, 46.70]}"`: set the region of interest using a bounding box defined by its [xmin, ymin, xmax, ymax] coordinates (in WGS84)
- `-i grid_model_build.ini`: use this ini file to configure the model build
- `-d artifact_data -d data/vito_reclass.yml`: parse the pre-defined artifact_data and the local vito_reclass data catalogs
- `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.



In [ ]:
print_dir("tmp_grid_model1")

In [ ]:
# checkout the content of the hypothetical model simulation configuration
fn_ini = "tmp_grid_model1/model.ini"
with open(fn_ini, "r") as f:
    txt = f.read()
print(txt)

### Update a model from CLI

Using the **hydromt update** method we can update an existing model with new components or modify existing components. Let's get an overview of the method and its arguments. 

Note that the `MODEL` (i.e. name of the model), and `MODEL_ROOT` (i.e. folder of existing model) are still required. There is an optional `-o --model-out` option to save the updated model in a different directory.

In [ ]:
!hydromt update --help

In this basic example we use the **hydromt update** method to update the GridModel instance with an upstream area raster map. Then we write only the updated model map component to file. 

The `%%writefile` magic saves the content below to a file. The content of the ini file start from the second line.

In [ ]:
%%writefile ./tmp_grid_model1/grid_model_update.ini 
[setup_maps_from_raster]
raster_fn = merit_hydro_1k
variables = uparea

[write_maps]

In [ ]:
!hydromt update grid_model ./tmp_grid_model1 -i ./tmp_grid_model1/grid_model_update.ini -vv

The example above means the following: run **hydromt update** with:

- `grid_model`: i.e. update a generic GridModel instance
- `./tmp_model1`: the folder of the to-be updated model
- `-i ./tmp_grid_model1/grid_model_update.ini`: the hydromt configuration listing the methods to be executed
- `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

In [ ]:
# note the difference with the original model
print_dir("tmp_grid_model1")